In [ ]:
import scvi
import os
os.environ['SLURM_JOB_NAME'] = 'bash'  
# https://github.com/PyTorchLightning/pytorch-lightning/issues/3651#issuecomment-796045752

import scanpy as sc
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np

sc.set_figure_params(figsize=(6,6))

In [ ]:
from pathlib import Path, PurePath

### read adata and subset to with only HVG

In [ ]:
#### snakemake input is a list
adata = sc.read(snakemake.input[0])

In [28]:
adata = adata[:,adata.var['highly_variable'] == True]
adata.obs['percent_mito'] = adata.obs['pct_counts_mt']/100

/var/folders/rm/svqn0lhx63z_94lkg6ttnsc80000gn/T/ipykernel_9596/3641185465.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['percent_mito'] = adata.obs['pct_counts_mt']/100


### model setup

#### test different seed

In [38]:
cat_covariate_list = ["patient"]
cont_covariate_list = ["percent_mito", "percent_ribo"]

In [44]:
def train_scvi(seed, adata, cat_covariate_list, cont_covariate_list, name):
    
    model_name = 'model' + time.strftime("_20%y%m%d_%H%M%S", time.localtime())
        
    np.random.seed(seed)
    
    # setup
    scvi.model.SCVI.setup_anndata(
    adata,
    layer="counts",
    categorical_covariate_keys=cat_covariate_list,
    continuous_covariate_keys=cont_covariate_list)
    
    # model
    model = scvi.model.SCVI(adata)
    
    # train
    model.train()
    
    # save
    print(model)
    if name:
        model_name = model_name + name
    print('model saving: ' + model_name )
    model.save(Path(model_name))

In [ ]:
train_scvi(0, adata, cat_covariate_list, None , name='_only_pt')

In [ ]:
train_scvi(0, adata, cat_covariate_list, [cont_covariate_list[1]] , name='_pt_ribo')